In [ ]:
import os

import networkx as nx
import torch

from pdmodels import ESMIF, MPNN, ReVor
from pdmodels.globals import ROOT_DIR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
checkpoint_path = os.path.join(
    ROOT_DIR, "model_params", "ligandmpnn", "ligandmpnn_v_32_020_25.pt"
)
mpnn = MPNN(
    checkpoint_path,
    model_type="ligand_mpnn",
    device=device,
    ligand_mpnn_use_side_chain_context=True,
)
# esmif = ESMIF(device=device)

In [ ]:
seqs_path = os.path.join(ROOT_DIR, "assets", "multimers", "multimers.fasta")
pdb_path = os.path.join(ROOT_DIR, "assets", "multimers", "NbALFA_ALFAtag_AF3.pdb")

seqs_wt = "SGEVQLQESGGGLVQPGGSLRLSCTASGVTISALNAMAMGWYRQAPGERRVMVAAVSERGNAMYRESVQGRFTVTRDFTNKMVSLQMDNLKPEDTAVYYCHVLEDRVDSFHDYWGQGTQVTVSS:PSRLEEELRRRLTEP"
repeat = 4
redesigned_residues = "A1 A3 A4 A5 A7 A8 A9 A13 A14 A15 A19 A20 A21 A23 A24 A25 A26 A27 A39 A41 A44 A45 A46 A48 A50 A52 A53 A67 A68 A69 A72 A73 A74 A75 A76 A77 A78 A79 A80 A81 A82 A83 A84 A85 A86 A88 A89 A91 A92 A93 A95 A97 A99 A100 A102 A114 A116 A118 A119 A120 A121 A123 A124"

In [ ]:
revor = ReVor(
    mpnn, pdb_path, seqs_wt, repeat=repeat, redesigned_residues=redesigned_residues
)
revor.revert(
    seqs_path,
    cutoff=0.1,
    batch_size=16,
    max_step=3,
    n_samples=6,
    temperature=1.0,
    checkpoint_path="./checkpoint.pkl",
    save_checkpoint_interval=2,
)

In [ ]:
revor.plot(nx.multipartite_layout, subset_key="iteration")
revor.plot(nx.multipartite_layout, subset_key="distance")
revor.plot(nx.multipartite_layout, subset_key="topology")

In [ ]:
revor.get_results()

In [ ]:
revor.save(os.path.join(ROOT_DIR, "assets", "multimers", "revor"))